In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE65106"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE65106"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE65106.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE65106.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE65106.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of skin fibroblast, iPSC, iPSC-derived neural progenitors, and iPSC-derived neurons from Autism Spectrum Disorder male patients and their unaffected normal male siblings"
!Series_summary	"Autism spectrum disorder (ASD) is an early onset neurodevelopmental disorder, which is characterized by disturbances of brain function and behavioral deficits in core areas of impaired reciprocal socialization, impairment in communication skills, and repetitive or restrictive interests and behaviors. ASD is known to have a significant genetic risk, but the underlying genetic variation can be attributed to hundreds of genes. The molecular and pathophysiologic basis of ASD remains elusive because of its genetic heterogeneity and complexity, its high comorbidity with other diseases, and the paucity of brain tissue for study. The invasive nature of collecting primary neuronal tissue from patients might be circumvented through reprogramming periph

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable, List, Tuple

# Analysis of dataset
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset contains gene expression data from microarray analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, we can identify:
# - Trait (Epilepsy): row 1 contains 'disease type'
# - Age: row 3 contains 'donor age'
# - Gender: row 4 contains 'donor sex'

trait_row = 1  # 'disease type: ASD' or 'disease type: Normal'
age_row = 3    # 'donor age: X'
gender_row = 4  # 'donor sex: Male' or 'donor sex: Female'

# 2.2 Data Type Conversion Functions
def convert_trait(val: str) -> int:
    """Convert trait value to binary (0: control, 1: case)"""
    if pd.isna(val):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in val:
        val = val.split(':', 1)[1].strip()
    
    # Convert ASD to 1 (case), Normal/WT to 0 (control)
    if val.lower() == 'asd':
        return 1
    elif val.lower() in ['normal', 'wt']:
        return 0
    return None

def convert_age(val: str) -> float:
    """Convert age value to continuous numeric value"""
    if pd.isna(val):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in val:
        val = val.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        # If it's 'embryonic', we can't assign a specific age
        if val.lower() == 'embryonic':
            return None
        return float(val)
    except (ValueError, TypeError):
        return None

def convert_gender(val: str) -> int:
    """Convert gender value to binary (0: female, 1: male)"""
    if pd.isna(val):
        return None
    
    # Extract value after colon and strip whitespace
    if ':' in val:
        val = val.split(':', 1)[1].strip()
    
    # Convert Male to 1, Female to 0
    if val.lower() == 'male':
        return 1
    elif val.lower() == 'female':
        return 0
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # First, convert the dictionary to a proper format
    sample_char_dict = {0: ['cell type: Fibroblast', 'cell type: hESC', 'cell type: iPSC', 'cell type: iPSC-derived NPC', 'cell type: iPSC-derived neuron'], 
                        1: ['disease type: ASD', 'disease type: Normal', 'disease type: WT'], 
                        2: ['donor id: AA1', 'donor id: AA2', 'donor id: AA3', 'donor id: AA4', 'donor id: AN1', 'donor id: AN2', 'donor id: AN3', 'donor id: AN4', 'donor id: NN1', 'donor id: NN2', 'donor id: NN3', 'donor id: CT2', 'donor id: ESI-053'], 
                        3: ['donor age: 8', 'donor age: 7', 'donor age: 9', 'donor age: 10', 'donor age: 16', 'donor age: embryonic'], 
                        4: ['donor sex: Male', 'donor sex: Female'], 
                        5: ['batch: 1a', 'batch: 2a', 'batch: 3a', 'batch: 3b', 'batch: 4a', 'cell line: CT2', 'cell line: ESI-053', 'batch: 9a', 'batch: 10a', 'batch: 12a', 'batch: 11a', 'batch: 21a', 'batch: 18a', 'batch: 19a', 'batch: 15a', 'batch: 16b', 'batch: 16a', 'batch: 19b', 'batch: 18b', 'batch: 17a', 'batch: 14d', 'batch: 14a', 'batch: 14c', 'batch: 13b', 'batch: 13a', 'batch: 14b'], 
                        6: [np.nan, 'batch: 7a', 'batch: 6a']}
    
    # Create DataFrame with sample characteristics
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{0: [1.0, 8.0, 1.0], 1: [0.0, 7.0, 0.0], 2: [0.0, 9.0, nan], 3: [nan, 10.0, nan], 4: [nan, 16.0, nan], 5: [nan, nan, nan], 6: [nan, nan, nan], 7: [nan, nan, nan], 8: [nan, nan, nan], 9: [nan, nan, nan], 10: [nan, nan, nan], 11: [nan, nan, nan], 12: [nan, nan, nan], 13: [nan, nan, nan], 14: [nan, nan, nan], 15: [nan, nan, nan], 16: [nan, nan, nan], 17: [nan, nan, nan], 18: [nan, nan, nan], 19: [nan, nan, nan], 20: [nan, nan, nan], 21: [nan, nan, nan], 22: [nan, nan, nan], 23: [nan, nan, nan], 24: [nan, nan, nan], 25: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE65106.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE65106/GSE65106_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE65106/GSE65106_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (33297, 59)
First 20 gene/probe identifiers:
['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe identifiers from an Illumina HumanHT-12 array, not standard human gene symbols.
# They need to be mapped to official gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Display a sample of the gene_assignment column which likely contains gene information
print("\nSample of gene_assignment column (first 3 rows):")
if 'gene_assignment' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['gene_assignment'].iloc[i][:200]}...")  # Show first 200 chars
    
    # Extract a sample of gene symbols from gene_assignment column
    print("\nAttempting to extract gene symbols from gene_assignment column...")
    
    # Function to extract gene symbols from gene_assignment string
    def extract_gene_symbol(gene_assign_str):
        if pd.isna(gene_assign_str) or gene_assign_str == '---':
            return None
        
        # Example pattern: "NM_001004195 // OR4F4 // olfactory receptor..."
        # We want to extract "OR4F4"
        symbols = []
        
        # Split by multiple gene entries (if any)
        gene_entries = gene_assign_str.split('///')
        
        for entry in gene_entries:
            parts = entry.split('//')
            if len(parts) >= 2:
                # The gene symbol is usually in the second part
                symbol_part = parts[1].strip()
                # Extract just the symbol (not the description)
                if symbol_part:
                    symbol = symbol_part.split()[0]
                    symbols.append(symbol)
        
        return symbols if symbols else None
    
    # Apply extraction to a sample
    sample_size = min(10, len(gene_annotation))
    sample_results = [extract_gene_symbol(gene_annotation['gene_assignment'].iloc[i]) 
                     for i in range(sample_size)]
    
    print(f"Extracted gene symbols from first {sample_size} rows:")
    for i, symbols in enumerate(sample_results):
        print(f"Row {i}: {symbols}")
    
    # Create mapping dataframe
    gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_gene_symbol)
    
    # Create the proper mapping format (ID to Gene)
    mapping_data = pd.DataFrame({
        'ID': np.repeat(gene_annotation['ID'].values, 
                        gene_annotation['Gene'].apply(lambda x: len(x) if isinstance(x, list) and x else 1)),
        'Gene': [symbol for symbols in gene_annotation['Gene'] 
                       for symbol in (symbols if isinstance(symbols, list) and symbols else [None])]
    })
    
    # Remove rows with None gene symbols
    mapping_data = mapping_data.dropna(subset=['Gene'])
    
    # Print sample of mapping data
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
    
    # Check how many unique gene symbols we have
    unique_genes = mapping_data['Gene'].nunique()
    print(f"Number of unique gene symbols: {unique_genes}")
else:
    print("\nError: Could not find 'gene_assignment' column in the annotation data")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '


Sample of probe ID to gene symbol mappings:
         ID     Gene
1   7896738   OR4G2P
2   7896738  OR4G11P
3   7896738   OR4G1P
4   7896740    OR4F4
5   7896740   OR4F17
6   7896740    OR4F5
7   7896740   OR4F17
8   7896740    OR4F4
9   7896740    OR4F5
10  7896740   OR4F17

Total number of probe-to-gene mappings: 323773
Number of unique gene symbols: 25760


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain the gene identifiers and gene symbols
# From the previous output, we can see:
# - The 'ID' column contains probe identifiers matching the gene expression data index
# - The 'gene_assignment' column contains gene symbol information

# 2. Create the gene mapping using our helper function
# First extract probe IDs and gene symbols from gene_annotation
prob_col = 'ID'
gene_col = 'gene_assignment'

# Create a mapping between probe IDs and gene symbols
mapping_df = gene_annotation[[prob_col, gene_col]].copy()
mapping_df = mapping_df.rename(columns={prob_col: 'ID'})

# Extract gene symbols from the gene_assignment column string
def extract_gene_symbols(gene_assign_str):
    """Extract gene symbols from gene_assignment string."""
    if pd.isna(gene_assign_str) or gene_assign_str == '---':
        return []
    
    # Use the extract_human_gene_symbols function from the library
    return extract_human_gene_symbols(gene_assign_str)

# Apply extraction to get gene symbols
mapping_df['Gene'] = mapping_df[gene_col].apply(extract_gene_symbols)
mapping_df = mapping_df[['ID', 'Gene']]  # Keep only ID and Gene columns

# Expand the mapping dataframe (one row per gene symbol)
mapping_df = mapping_df.explode('Gene')
mapping_df = mapping_df.dropna(subset=['Gene'])  # Remove rows with no gene symbol

# 3. Load gene expression data and apply the mapping
# First, load the genetic data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_expr_data = get_genetic_data(matrix_file)

# Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_expr_data, mapping_df)

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)

# Preview the gene expression data
print(f"Original gene expression data shape: {gene_expr_data.shape}")
print(f"Gene mapping shape: {mapping_df.shape}")
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())
print("\nPreview of gene expression data (first 5 rows, first 5 columns):")
if gene_data.shape[1] >= 5:
    print(gene_data.iloc[:5, :5])
else:
    print(gene_data.iloc[:5, :])

# Save the gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Update the gene availability flag based on the results
is_gene_available = gene_data.shape[0] > 0 and gene_data.shape[1] > 0
print(f"Gene data available: {is_gene_available}")


Original gene expression data shape: (33297, 59)
Gene mapping shape: (155269, 2)
Mapped gene expression data shape: (24235, 59)
First 10 gene symbols:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']

Preview of gene expression data (first 5 rows, first 5 columns):
         GSM1587362  GSM1587363  GSM1587364  GSM1587365  GSM1587366
Gene                                                               
A1BG        6.93874     7.06714     6.79534     7.04644     6.73879
A1CF        4.64016     4.68643     4.61005     4.73412     4.41252
A2M         5.73310     5.21556     5.66245     5.75909     5.29572
A2ML1       4.99052     5.23956     4.79872     5.19624     5.10726
A3GALT2     5.64074     5.31316     5.53539     5.72826     5.32214


Gene expression data saved to ../../output/preprocess/Epilepsy/gene_data/GSE65106.csv
Gene data available: True


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (24235, 59)
Gene data shape after normalization: (24235, 59)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE65106.csv
Extracted clinical data shape: (3, 59)
Preview of clinical data (first 5 samples):
          GSM1587362  GSM1587363  GSM1587364  GSM1587365  GSM1587366
Epilepsy         1.0         1.0         1.0         1.0         1.0
Age              8.0         8.0         7.0         7.0         9.0
Gender           1.0         1.0         1.0         1.0         1.0
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE65106.csv
Gene data columns (first 5): ['GSM1587362', 'GSM1587363', 'GSM1587364', 'GSM1587365', 'GSM1587366']
Clinical data columns (first 5): ['GSM1587362', 'GSM1587363', 'GSM1587364', 'GSM1587365', 'GSM1587366']
Found 59 common samples between gene and clinical data
Initial linked data shape: (59, 24238)
Preview of linked data (first 5 rows, first 5 columns):
            Epilepsy  Age  Gender     A1BG     A1CF
GSM1587362       1.0  8.0     1.0  6.93874  4.64016
GSM1587363    

Linked data shape after handling missing values: (59, 24238)
For the feature 'Epilepsy', the least common label is '1.0' with 21 occurrences. This represents 35.59% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 7.0
  50% (Median): 9.0
  75%: 10.0
Min: 7.0
Max: 16.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 4 occurrences. This represents 6.78% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Epilepsy/GSE65106.csv
